# Problèmes

### 1)
Primal: $\underset{x \in \mathbb{R}^3}{\min} x_1 ~~ s.t. ~ x_1 + x_2 + x_3 = 1 ~, ~~ x \ge 0$

Dual: $\underset{\lambda \in \mathbb{R}, s \in \mathbb{R^3}}{\max} \lambda ~~ s.t. ~ [1~1~1]' \lambda + s = [1~0~0]'~, ~~s\ge0$

### 2)
Primal : $\underset{x \in \mathbb{R}^3}{\min} -2 x_1 - 3 x_2 ~~ s.t. ~ \begin{bmatrix}
1 & 1 & 1 & 0\\
2 & 1 & 0 & 1
\end{bmatrix} 
\begin{bmatrix}
x_1 \\ x_2 \\ x_3 \\ x_4 \end{bmatrix} = \begin{bmatrix} 50 \\ 30 \end{bmatrix} ~, ~~ x \ge 0$

In [120]:
using LinearAlgebra
using LaTeXStrings
using DataStructures
using BenchmarkTools

In [121]:
function display_results(result)
    # fonction pour l'affichage
    println("\n-----------------------------------------------------------------------")
    println("------------------------------- RESULTS -------------------------------")
    result
end


display_results (generic function with 1 method)

In [122]:

function Point_in_Neighborhood_infty(gamma, beta, A, b, c, x, lambda, s, mu, x_0, lambda_0, s_0, mu_0)
    
    rb = A*x - b
    rb0 = A*x_0 - b
    rc = A'*lambda + s - c
    rc0 = A'*lambda_0 + s_0 - c
    xs_pv = x .* s
    
    # tester conditions sur norm([rb, rc]) ou sur norm(rb) et norm(rc indépendemment) ?
     return sum(x.<=0)==false && sum(s.<=0)==false && sum(xs_pv.<=gamma*mu)==false &&
          norm(rb) <= norm(rb0)*beta*mu/mu_0 && norm(rc) <= norm(rc0)*beta*mu/mu_0
    #return sum(x.<=0)==false && sum(s.<=0)==false && sum(xs_pv.<=gamma*mu)==false &&
    #      norm(vcat(rb, rc)) <= norm(vcat(rb0, rc0))*beta*mu/mu_0 
end

function Next_Point_in_Neighborhood_infty(alpha_step, gamma, beta, A, b, c, x, lambda, 
                                        s, dir_k, mu, x_0, lambda_0, s_0, mu_0, n, m)
    x_next = x + alpha_step*dir_k[1:n]
    lambda_next = lambda + alpha_step*dir_k[n+1: n+m]
    s_next = s + alpha_step*dir_k[n+m+1: end]
    mu_next = s_next' * x_next / n
    return Point_in_Neighborhood_infty(gamma, beta, A, b, c, x_next, lambda_next, s_next, mu_next, x_0, lambda_0, s_0, mu_0)
end

function Next_Point_ArmijoCond(alpha, alpha_step, x, s, dir_k, mu, n, m)
    x_next = x + alpha_step*dir_k[1:n]
    s_next = s + alpha_step*dir_k[n+m+1: end]
    mu_next = s_next' * x_next / n
    return mu_next <= (1 - 0.001 * (alpha+alpha_step)) * mu
end

function UpdateAlphaOpt(gamma, beta, A, b, c, x, lambda, s, dir_k, mu, x_0, lambda_0, s_0, mu_0, alpha_step)
    alpha = 0
    x_k, lambda_k, s_k = x, lambda, s
    n = length(x)
    m = length(lambda)
    while Next_Point_in_Neighborhood_infty(alpha_step, gamma, beta,  A, b, c, x_k,lambda_k, 
            s_k, dir_k, mu, x_0, lambda_0, s_0, mu_0, n, m) == true && alpha+alpha_step <= 1 &&
            Next_Point_ArmijoCond(alpha, alpha_step, x_k, s_k, dir_k, mu, n, m) == true
    
        alpha += alpha_step
        x_k = x_k + alpha_step*dir_k[1:n]
        lambda_k = lambda_k + alpha_step*dir_k[n+1: n+m]
        s_k = s_k + alpha_step*dir_k[n+m+1: end]
    end
        
    return x_k, lambda_k, s_k        
end

UpdateAlphaOpt (generic function with 2 methods)

In [123]:

function InfeasiblePathFollowing1(c, A, b, init_vect, gamma, beta, sigma_min, sigma_max, 
    max_iter, eps = 1e-8, alpha_step=1e-2, display=true)

    x_0, lambda_0, s_0 = init_vect
    x_k, lambda_k, s_k = copy(x_0), copy(lambda_0), copy(s_0)
    n_rows, n_cols = size(A) 

    rb_0 = A * x_0 - b
    rc_0 = A' * lambda_0 + s_0 - c
    mu_0 = s_0' * x_0 / n_cols
    Xk = Diagonal(x_k)
    Sk = Diagonal(s_k)
    mu_k = s_k' * x_k / n_cols
    rb_k = copy(rb_0)
    rc_k = copy(rc_0)
    
    k = 0
    pdd, cond_rb, cond_rc = 1, 1, 1
    e = ones(n_cols, 1)
    n_c = norm(c)
    n_b = norm(b)
    
    while k <= max_iter && pdd > eps && cond_rb > eps && cond_rc > eps  # 3 epsilon différents en pratique???

        sigma_k = sigma_min  # comment choisir sigma en pratique? ici je prends toujours sigma_min
        
        Jacob_Fk = [zeros(n_cols, n_cols) A' I(n_cols)
                    A zeros(n_rows, n_rows) zeros(n_rows, n_cols)
                    Sk zeros(n_cols, n_rows) Xk]
        Fk = [-rc_k
                -rb_k
                -Xk*Sk*e + sigma_k*mu_k*e]

        dir_k = Jacob_Fk\Fk

        x_k, lambda_k, s_k = UpdateAlphaOpt(gamma, beta, A, b, c, x_k, lambda_k, s_k, dir_k,
                                            mu_k, x_0, lambda_0, s_0, mu_0, alpha_step)
        
        Xk = Diagonal(x_k)
        Sk = Diagonal(s_k)
        mu_k = s_k' * x_k / n_cols
        rb_k = A * x_k - b
        rc_k = A' * lambda_k + s_k - c
        
        # valeurs pour les conditions d'arret:
        pdd = abs(c' * x_k - b' * lambda_k) / (1 + abs(c' * x_k))
        cond_rb = norm(rb_k) / (1 + n_b)
        cond_rc = norm(rc_k) / (1 + n_c)
        
        k += 1
        
        # affichage
        if display == true
            println("---------------------------------------------------------------")
            println("Iter ", k, ":   pdd = ", pdd, " |  cond_rb = ", cond_rb, " |  cond_rc = ", cond_rc)
        end
    end
    
    if display == true
        criteria = [k > max_iter,  pdd <= eps, cond_rb <= eps, cond_rc <= eps]
        criteria_names = ["reached max_iter",  "pdd <= eps", "cond_rb <= eps", "cond_rc <= eps"]
        println("\n stopping criterion = ",criteria_names[findall(criteria)])
    end
    return OrderedDict("x_opt" => x_k, "lambda_opt" => lambda_k, "s_opt" => s_k, 
        "n_iter" => k, "pdd" => pdd, "cond_rb" => cond_rb, "cond_rc" => cond_rc)
end

InfeasiblePathFollowing1 (generic function with 4 methods)

In [124]:
### probleme 1
c = [1, 0 ,0]
A = Matrix([1, 1, 1]')
b = [1];

In [125]:
# initialisation 1 
x01 = [10, 4, 6]
lambda01 = [-2]
s01 = [3, 4, 2]
init_vect1 = (x01, lambda01, s01);
gamma, beta = 0.3, 2  # pour ce problème, ne marche pas lorque gamma est trop grand.
res_ipf1 =  InfeasiblePathFollowing1(c, A, b, init_vect1, gamma, beta, 0.2, 0.3, 1000);
display_results(res_ipf1) 

---------------------------------------------------------------
Iter 1:   pdd = 2.1798114585407875 |  cond_rb = 1.2350000000000179 |  cond_rc = 0.12999999999999923
---------------------------------------------------------------
Iter 2:   pdd = 2.283136859110721 |  cond_rb = 0.012350000000001193 |  cond_rc = 0.0013000000000134015
---------------------------------------------------------------
Iter 3:   pdd = 0.5405400488468726 |  cond_rb = 0.00012349999999727856 |  cond_rc = 1.2999999996987643e-5
---------------------------------------------------------------
Iter 4:   pdd = 0.12211384100798554 |  cond_rb = 1.2350000015981522e-6 |  cond_rc = 1.2999999904686477e-7
---------------------------------------------------------------
Iter 5:   pdd = 0.027046971846905153 |  cond_rb = 1.2349998579352928e-8 |  cond_rc = 1.299999874244071e-9

 stopping criterion = ["cond_rc <= eps"]

-----------------------------------------------------------------------
------------------------------- RESULTS ----

OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [0.0118487, 0.565947, 0.422204]
  "lambda_opt" => [-0.0155188]
  "s_opt"      => [1.01552, 0.0155188, 0.0155188]
  "n_iter"     => 5
  "pdd"        => 0.027047
  "cond_rb"    => 1.235e-8
  "cond_rc"    => 1.3e-9

In [126]:
# cond_rb et cond_rc trouvées dans le livre mais elles semblent arrêter très rapidement l'algorithme...

In [127]:
### probleme 2
c2 = [-2; -3; 0; 0]
A2 = [1 1 1 0
      2 1 0 1]
b2 = [50; 30];

In [128]:
# initialisation 
x02 = [4; 1; 3; 2]
lambda02 = [-2; 3]
s02 = [3; 4; 2; 3]
init_vect2 = (x02, lambda02, s02);
gamma, beta = 0.3, 2
res_ipf2 =  InfeasiblePathFollowing1(c2, A2, b2, init_vect2,  gamma, beta, 0.2, 0.3, 1000);
display_results(res_ipf2)

---------------------------------------------------------------
Iter 1:   pdd = 4.268415615872338 |  cond_rb = 0.7150606724766477 |  cond_rc = 2.6874815593190524
---------------------------------------------------------------
Iter 2:   pdd = 4.0507764473276975 |  cond_rb = 0.6578558186785161 |  cond_rc = 2.472483034573528
---------------------------------------------------------------
Iter 3:   pdd = 3.764118954431787 |  cond_rb = 0.6249630277445903 |  cond_rc = 2.3488588828448513
---------------------------------------------------------------
Iter 4:   pdd = 3.3201093165953326 |  cond_rb = 0.5749659855250231 |  cond_rc = 2.1609501722172633
---------------------------------------------------------------
Iter 5:   pdd = 2.515061591198974 |  cond_rb = 0.4772217679857691 |  cond_rc = 1.7935886429403276
---------------------------------------------------------------
Iter 6:   pdd = 1.851515345253503 |  cond_rb = 0.3960940674281884 |  cond_rc = 1.4886785736404717
---------------------------

OrderedDict{String,Any} with 7 entries:
  "x_opt"      => [6.21382e-5, 29.9998, 20.0001, 8.28467e-5]
  "lambda_opt" => [-1.24289e-5, -3.0]
  "s_opt"      => [4.0, 8.28395e-6, 1.24289e-5, 3.0]
  "n_iter"     => 24
  "pdd"        => 1.09244e-5
  "cond_rb"    => 3.2359e-10
  "cond_rc"    => 1.21618e-9

In [129]:
@benchmark res_ipf2 =  InfeasiblePathFollowing1(c2, A2, b2, init_vect2,  gamma, beta, 0.2, 0.3, 100, 1e-10, 1e-2, false )

BenchmarkTools.Trial: 
  memory estimate:  3.05 MiB
  allocs estimate:  31818
  --------------
  minimum time:     1.361 ms (0.00% GC)
  median time:      1.385 ms (0.00% GC)
  mean time:        1.626 ms (11.81% GC)
  maximum time:     6.362 ms (62.68% GC)
  --------------
  samples:          3068
  evals/sample:     1